In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define hyperparameters
num_classes = 2
batch_size = 32
epochs = 10

# Load the pre-trained InceptionResNetV2 model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer and a fully connected layer on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Use ImageDataGenerator to preprocess and augment the training, validation, and test images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    'val',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    'test',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)


2023-02-16 06:25:05.711546: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64::/opt/conda/lib
2023-02-16 06:25:05.711626: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


219055592/219055592 [==============================] - 10s 0us/step
Found 24588 images belonging to 2 classes.
Found 5269 images belonging to 2 classes.
Found 5269 images belonging to 2 classes.
Epoch 1/10
769/769 [==============================] - 3938s 5s/step - loss: 0.5844 - accuracy: 0.7363 - val_loss: 0.5531 - val_accuracy: 0.7375
Epoch 2/10
769/769 [==============================] - ETA: 0s - loss: 0.5540 - accuracy: 0.7412

In [1]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils

# Set up the directory containing the image data
data_dir = '/kaggle/input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images'
classes = ['No_DR', 'Mild', 'Moderate', 'Severe', 'Proliferate_DR']

# Set up the input image dimensions
img_rows, img_cols = 256, 256

# Set up empty lists to store the image data and labels
data = []
labels = []

# Load the image data and perform segmentation
for c in classes:
    path = os.path.join(data_dir, c)
    class_num = classes.index(c)
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_rows, img_cols))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
        data.append(thresh)
        labels.append(class_num)

# Convert the data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Reshape the data to match the input dimensions of the model
data = data.reshape((-1, img_rows, img_cols, 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Normalize the input data
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Convert the labels to categorical format
y_train = np_utils.to_categorical(y_train, len(classes))
y_test = np_utils.to_categorical(y_test, len(classes))

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_rows, img_cols, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), target_names=classes))

Epoch 1/10
879/879 [==============================] - 791s 898ms/step - loss: 1.0758 - accuracy: 0.7253 - val_loss: 0.8660 - val_accuracy: 0.7327
Epoch 2/10
879/879 [==============================] - 744s 847ms/step - loss: 0.8529 - accuracy: 0.7353 - val_loss: 0.8590 - val_accuracy: 0.7327
Epoch 3/10
879/879 [==============================] - 745s 847ms/step - loss: 0.7595 - accuracy: 0.7391 - val_loss: 0.8743 - val_accuracy: 0.7327
Epoch 4/10
879/879 [==============================] - 739s 841ms/step - loss: 0.6414 - accuracy: 0.7580 - val_loss: 0.9782 - val_accuracy: 0.7314
Epoch 5/10
879/879 [==============================] - 735s 836ms/step - loss: 0.5240 - accuracy: 0.7923 - val_loss: 1.1788 - val_accuracy: 0.7301
Epoch 6/10
879/879 [==============================] - 734s 835ms/step - loss: 0.4357 - accuracy: 0.8248 - val_loss: 1.3339 - val_accuracy: 0.7213
Epoch 7/10
879/879 [==============================] - 742s 844ms/step - loss: 0.3734 - accuracy: 0.8480 - val_loss: 1.4203 -